# Тестовое задание
Информация о данных: https://ai.stanford.edu/~amaas/data/sentiment/

Короткое объяснение: требуется обучить модель на задачу классификации комментариев пользователей к фильмам, а также сделать возможным интерпретировать ответ в значения от 1 до 10 (например, звезд).

In [464]:
import math
import random
import string
import os
import re

import numpy as np
import pandas as pd
import seaborn as sns

from catboost import CatBoostClassifier
from gensim.corpora import Dictionary
from gensim.models import Word2Vec
from gensim.utils import tokenize

import sklearn
import torch
import nltk
import gensim
import gensim.downloader as api

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Даня\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Считаем данные

In [13]:
TRAIN_DATA_PATH = 'data/aclImdb/train/'
TEST_DATA_PATH = 'data/aclImdb/test/'

In [558]:
def open_txt(path):
    comments_names = os.listdir(path)
    if path[-4:] == 'pos/':
        target = 1
    else: 
        target = 0
    comments = []
    for comment in comments_names:
        with open(os.path.join(path, comment) , encoding="utf8") as f:
            lines = f.readlines()
            comments.append([lines[0], target])
    return comments

In [559]:
train_pos = open_txt('data/aclImdb/train/pos/')
train_neg = open_txt('data/aclImdb/train/neg/')
test_pos = open_txt('data/aclImdb/test/pos/')
test_neg = open_txt('data/aclImdb/test/neg/')

In [560]:
#словарь
with open('data/aclImdb/imdb.vocab', encoding="utf8") as f:
        all_words = f.readlines()
        all_words[i] = all_words[i][:-1]
for i in range(len(all_words)):
    all_words[i] = all_words[i][:-1]

In [561]:
train_df = pd.DataFrame(train_pos + train_neg, columns = ['text', 'target'])
train_df.head(3)

,text,target
0,Bromwell High is a cartoon comedy. It ran at t...,1
1,Homelessness (or Houselessness as George Carli...,1
2,Brilliant over-acting by Lesley Ann Warren. Be...,1


In [562]:
test_df = pd.DataFrame(test_pos + test_neg, columns = ['text', 'target'])
test_df.head(3)

,text,target
0,I went and saw this movie last night after bei...,1
1,Actor turned director Bill Paxton follows up h...,1
2,As a recreational golfer with some knowledge o...,1


# Токенизируем данные

In [563]:
punct_symbols = '"!@#$%^&*():;-+?_=,<>/'
def tokenizer_text(sent):
    tokens = [word.lower() for word in nltk.word_tokenize(sent)]
    tokens = [word for word in tokens if word not in pucnt_symbols]
    return tokens

In [564]:
train_df['tokenized'] = train_df['text'].apply(tokenizer_text)
test_df['tokenized'] = test_df['text'].apply(tokenizer_text)

In [565]:
test_df

,text,target,tokenized
0,I went and saw this movie last night after bei...,1,"[i, went, and, saw, this, movie, last, night, ..."
1,Actor turned director Bill Paxton follows up h...,1,"[actor, turned, director, bill, paxton, follow..."
2,As a recreational golfer with some knowledge o...,1,"[as, a, recreational, golfer, with, some, know..."
3,"I saw this film in a sneak preview, and it is ...",1,"[i, saw, this, film, in, a, sneak, preview, an..."
4,Bill Paxton has taken the true story of the 19...,1,"[bill, paxton, has, taken, the, true, story, o..."
...,...,...,...
24995,I occasionally let my kids watch this garbage ...,0,"[i, occasionally, let, my, kids, watch, this, ..."
24996,When all we have anymore is pretty much realit...,0,"[when, all, we, have, anymore, is, pretty, muc..."
24997,The basic genre is a thriller intercut with an...,0,"[the, basic, genre, is, a, thriller, intercut,..."
24998,Four things intrigued me as to this film - fir...,0,"[four, things, intrigued, me, as, to, this, fi..."


In [566]:
dictionary = Dictionary(train_df['tokenized'])
dic = dictionary.token2id
inv_dic = {v: k for k, v in dic.items()}

In [567]:
len(dictionary)

111703

# Обертка на Word2Vec

In [568]:
class Word2VecWrap:
    def __init__(self, tokens, vec_size, window, min_count, epochs):
        self.tokens = tokens
        self.epochs = epochs
        self.vec_size = vec_size
        self.model_wv = Word2Vec(sentences=tokens,
                                 vector_size=vec_size,
                                 window=window,
                                 min_count=min_count)

    def train(self):
        self.model_wv.train(tokens, total_examples=len(tokens), epochs=self.epochs)

    def save(self, path):
        self.model_wv.save(path)

    def vocab(self):
        return self.model_wv.wv

# Обучаем и проверяем Word2Vec
Получать векторное представление предложения будем с помощью w2v, где эмбеддинги всех слов в предложении просто будем усреднять

In [569]:
model_wv = Word2VecWrap(train_df['tokenized'], vec_size=64, window=5, min_count=3, epochs=15)
model_wv.train()

In [570]:
len(model_wv.vocab()) #если бы min_count = 1, то слов бы было 115к

42788

In [571]:
def sent2vec_train(question, model):
    token_question = np.array(question)
    question_array = np.zeros(model.vec_size)
    count = 0
    for word in token_question:
        if model.vocab().__contains__(str(word)):
            question_array += (np.array(model.vocab()[str(word)]))
            count += 1
    if count == 0:
        return question_array

    return question_array / count

In [572]:
def cos_sim(a, b):
    similarity = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
    return similarity

In [573]:
#Как мы видим word2vec работает, ембеддинговое пространство он построил 
mom = sent2vec_train(['mom'], model_wv)
dad = sent2vec_train(['dad'], model_wv)
fire = sent2vec_train(['fire'], model_wv)
print("Косинусное расстояние между словами мама и папа: ", cos_sim(mom, dad), '\n',
     "Косинусное расстояние между словами мама и огонь: ", cos_sim(mom, fire), sep='')

pos_1 = sent2vec_train(train_df['tokenized'][100], model_wv)
pos_2 = sent2vec_train(train_df['tokenized'][200], model_wv)
print("Косинусное расстояние между pos предложениями: ", cos_sim(pos_1, pos_2))

Косинусное расстояние между словами мама и папа: 0.9359934025562218
Косинусное расстояние между словами мама и огонь: 0.26493216103160466
Косинусное расстояние между pos предложениями:  0.8927138205559247


# Пишем обертку на CatBoost
Будем подавать эмбеддинги предложений в катбуст для предсказания таргета

In [574]:
class CatBoostClassifierWrap:
    def __init__(self, n_estimators, max_depth, learning_rate):
        self.model = CatBoostClassifier(n_estimators=n_estimators,
                                   max_depth=max_depth,
                                   learning_rate=learning_rate,
                                   early_stopping=50,
                                   random_state=42,
                                   verbose=1)

    def fit(self, x_train, y_train):
        self.model.fit(x_train, y_train)

    def save(self, path):
        pickle.dump(self.model, open(path, "wb"))

    def predict(self, x):
        return self.model.predict(x)

    def predict_proba(self, x):
        return self.model.predict_proba(x)

In [575]:
def data_to_model(df):
    X, Y = [], []
    for sentence in range(len(df['target'])):
        X.append(sent2vec_train(df['tokenized'][sentence], model_wv))
        Y.append(df['target'][sentence])
    return np.array(X), np.array(Y)

In [612]:
assert len(sent2vec_train(train_df['tokenized'][0], model_wv)) == 64

In [577]:
X_train, Y_train = data_to_model(train_df)
X_test, Y_test = data_to_model(test_df)

In [605]:
model_cl = CatBoostClassifier(max_depth=6, learning_rate=0.10, verbose=100)
model_cl.fit(X_train, Y_train)

0:	learn: 0.6766503	total: 11.3ms	remaining: 11.3s
100:	learn: 0.4650042	total: 1.03s	remaining: 9.2s
200:	learn: 0.4046956	total: 2.07s	remaining: 8.22s
300:	learn: 0.3618545	total: 3.11s	remaining: 7.22s
400:	learn: 0.3268324	total: 4.12s	remaining: 6.16s
500:	learn: 0.2973260	total: 5.14s	remaining: 5.12s
600:	learn: 0.2708515	total: 6.15s	remaining: 4.08s
700:	learn: 0.2476563	total: 7.16s	remaining: 3.05s
800:	learn: 0.2262150	total: 8.17s	remaining: 2.03s
900:	learn: 0.2076009	total: 9.19s	remaining: 1.01s
999:	learn: 0.1906062	total: 10.2s	remaining: 0us


In [606]:
model_cl.predict_proba(X_train[2])

array([0.2441681, 0.7558319])

# Оценка результатов
Посмотрим на accuracy, recall, precision на трейне и на тесте

In [607]:
train_predicts = model_cl.predict(X_train)
print('Accuracy: ', round((train_predicts == Y_train).sum() / len(Y_train), 4), '\n',
      'Precision: ', round(sklearn.metrics.precision_score(Y_train, train_predicts), 4), '\n',
      'Recall: ', round(sklearn.metrics.recall_score(Y_train, train_predicts), 4), sep='')

Accuracy: 0.9629
Precision: 0.9651
Recall: 0.9605


In [608]:
test_predicts = model_cl.predict(X_test)
print('Accuracy: ', round((test_predicts == Y_test).sum() / len(Y_test), 4), '\n',
      'Precision: ', round(sklearn.metrics.precision_score(Y_test, test_predicts), 4), '\n',
      'Recall: ', round(sklearn.metrics.recall_score(Y_test, test_predicts), 4), sep='')

Accuracy: 0.7882
Precision: 0.7882
Recall: 0.7882


# Выводы, идеи, предложения
### Это ноутбук показывает ML/DL часть, в train.py/model.py будет меньше кода, т.к. тут присутствует небольшой анализ. Оценку комментарию по итогу будем ставить смотря на вероятности, которые нам возвращает модель, то есть если вероятность pos = 0.84, то будем считать, что пользователь поставил 8/10
### Видим, что модель переобучилась, с этим можно бороться разными способами, один из самых главных - тюнинг параметров, причем не только самого бустинга, но и w2v, размерности векторов(ее можно сжать с помощью SVD-разложения) и прочих параметров.
### Также было неколько идей: взять трансформер с HF, взять tf-idf или doc2vec(т.к. комментарии достаточно длинные), можно было попробовать RNN и предсказывать таргет сразу одной моделью, а не как в моем решении (сначала получаем ембеддинговое представление word2vec-а, а потом передаем в catboost).
### Вариантов очень много, можно было бы использовать даже k-means или DBSCAN, но все пробовать в данном случае смысла не имеет, это тестовое задание и один и вариантов его решения